In [0]:
%run "../includes/configurations"


In [0]:
race_results_df=spark.read.parquet("/mnt/formula1dljit/presentation/race_results")


In [0]:
from pyspark.sql.functions import *

In [0]:
constructor_standing_df=race_results_df.groupBy("race_year","team").\
agg(sum(col("points")).alias("total_points"),
    count(when(col("positionOrder")==1,1).alias("wins")).alias("wins")
    ).orderBy(col("race_year").desc(),col("total_points").desc())

In [0]:
from pyspark.sql.window import Window

In [0]:
rank_spec=Window.partitionBy("race_year").orderBy(col("total_points").desc(),col("wins").desc())


In [0]:
final_df=constructor_standing_df.withColumn("rank",rank().over(rank_spec))

In [0]:
display(final_df.filter("race_year=2020"))

In [0]:
#final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/constructor_standing")

In [0]:
final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.constructor_standing")